In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/extracted-music-data/music_data.pkl
/kaggle/input/music-gen-model/tensorflow2/default/1/music_gen_model.keras


# 1. IMPORTS

In [2]:
import numpy as np
from tensorflow import keras
from music21 import instrument, note, chord, stream
import pickle
import random

2025-11-14 11:30:59.131875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763119859.337413      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763119859.396181      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# 2. LOAD PREPROCESSED DATA (.pkl)

In [3]:
print("Loading preprocessed data...")

with open("/kaggle/input/extracted-music-data/music_data.pkl", "rb") as f:
    data = pickle.load(f)

notes = data["notes"]
unique_notes = data["unique_notes"]
note_to_int = data["note_to_int"]

# Reverse mapping
int_to_note = {i: n for n, i in note_to_int.items()}

print(f"Loaded {len(notes)} notes.")
print(f"Unique notes: {len(unique_notes)}")

# Create sequences the same way model was trained
sequence_length = 100
network_input = []

for i in range(0, len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    network_input.append([note_to_int[n] for n in seq_in])

n_vocab = len(unique_notes)
print(f"Total sequences: {len(network_input)}")

Loading preprocessed data...
Loaded 1816494 notes.
Unique notes: 2652
Total sequences: 1816394


# 3. LOAD TRAINED MODEL (.keras)

In [6]:
print("Loading model...")
model = keras.models.load_model("/kaggle/input/music-gen-model/tensorflow2/default/1/music_gen_model.keras")
print("Model loaded successfully!")

Loading model...


I0000 00:00:1763121823.078961      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763121823.079672      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model loaded successfully!


# 4. GENERATE MUSIC

In [7]:
num_generate = 500  # number of notes to generate
print(f"\nGenerating {num_generate} notes...")

start_index = random.randint(0, len(network_input) - 1)
pattern = network_input[start_index]  # starting seed sequence
prediction_output = []

for i in range(num_generate):
    # reshape and normalize input
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    # predict next note
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    # update pattern
    pattern.append(index)
    pattern = pattern[1:]

print("\nGeneration complete!")


Generating 500 notes...


I0000 00:00:1763121827.233400     116 cuda_dnn.cc:529] Loaded cuDNN version 90300



Generation complete!


# 5. CONVERT GENERATED OUTPUT TO MIDI

In [8]:
print("Converting to MIDI...")

output_notes = []
offset = 0  # time shift between notes

for pattern in prediction_output:
    # chord
    if ("." in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split(".")
        chord_notes = []
        for n in notes_in_chord:
            new_note = note.Note(int(n))
            new_note.storedInstrument = instrument.Piano()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.storedInstrument = instrument.Piano()
        new_note.offset = offset
        output_notes.append(new_note)

    offset += 0.5  # spacing between notes

midi_stream = stream.Stream(output_notes)
midi_stream.write("midi", fp="output.mid")

print("\n🎶 DONE! Your file is saved as: output.mid")

Converting to MIDI...

🎶 DONE! Your file is saved as: output.mid
